Clean Code

In [1]:
# library needed
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
from sklearn import preprocessing
import scipy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import time
import pickle
import copy

# Code to deal with salary column

In [2]:
# see if character is in text
def alpha_in_text(text):
    return(any(c.isalpha() for c in text))

# see how many dashes are in text
def number_of_dashes(text):
    return(sum([1 for i in text if '-' in i]))

# extract smallest salary range value
def salary_extract_first(text):
    
    if pd.isna(text) is True:
        return(-1)
    
    elif alpha_in_text(text) is True:
        return(-2)
    
    elif '-' in text:
        if number_of_dashes(text) == 1:
            if re.split('-',text)[0].isdigit() is True:
                return(float(re.split('-',text)[0]))
            else:
                return(-1)
            
        else:
            return(-1)
    else:
        return(-1)
    
# largest salary range value
def salary_extract_second(text):
    
    if pd.isna(text) is True:
        return(-1)
    
    elif alpha_in_text(text) is True:
        return(-2)
    
    elif '-' in text:
        if number_of_dashes(text) == 1:
            if re.split('-',text)[1].isdigit() is True:
                return(float(re.split('-',text)[1]))
            else:
                return(-1)
            
        else:
            return(-1)
    else:
        return(-1)

# convert numeric salary to category
def salary_category_first(number):
    percentile = [60.0, 14000.0, 20000.0, 30000.0, 35000.0, 44374.4, 55000.0, 70000.0, 90000.0]
    if number == -1:
        return(str(1))
    
    if number == -2:
        return(str(2))
    
    for i in range(len(percentile)):
        if i not in {0,8}:
            if (number > percentile[i-1]) & (number <= percentile[i]):
                return(str(i+3))
            else:
                continue
            
        if i == 0:
            if number < percentile[0]:
                return(str(i+3))
        if i == 8:
            if number >= percentile[8]:
                return(str(i+3))
            


def salary_category_second(number):
    percentile = [120, 20000.0, 30000.0, 40000.0, 50000.0, 65000.0, 80000.0, 100000.0, 130000.0]
    if number == -1:
        return(str(1))
    
    if number == -2:
        return(str(2))
    
    for i in range(len(percentile)):
        if i not in {0,8}:
            if (number > percentile[i-1]) & (number <= percentile[i]):
                return(str(i+3))
            else:
                continue
            
        if i == 0:
            if number < percentile[0]:
                return(str(i+3))
        if i == 8:
            if number >= percentile[8]:
                return(str(i+3))

# Read Training Data

In [3]:
dat = pd.read_csv("job_training_data.csv")
stop_words = {'a', 'the', 'of', 'in', 'for'}

# Models to train on training data

In [4]:
vectorizer = CountVectorizer(stop_words=stop_words).fit(dat[['company_profile','description','requirements','benefits']].fillna('NaN').agg(' '.join,axis=1))
# one hot encoding
employment_type_onehot = OneHotEncoder(handle_unknown='ignore').fit(dat[['employment_type']].fillna('NaN'))
required_experience_onehot = OneHotEncoder(handle_unknown='ignore').fit(dat[['required_experience']].fillna('NaN'))
required_education_onehot = OneHotEncoder(handle_unknown='ignore').fit(dat[['required_education']].fillna('NaN'))
industry_onehot = OneHotEncoder(handle_unknown='ignore').fit(dat[['industry']].fillna('NaN'))
function_onehot = OneHotEncoder(handle_unknown='ignore').fit(dat[['function.']].fillna('NaN'))
category_1 = dat.salary_range.apply(salary_extract_first).apply(salary_category_first)
category_2 = dat.salary_range.apply(salary_extract_second).apply(salary_category_second)
salary_1_onehot = OneHotEncoder(handle_unknown='ignore').fit(pd.DataFrame(category_1))
salary_2_onehot = OneHotEncoder(handle_unknown='ignore').fit(pd.DataFrame(category_2))

# Parsing

In [5]:
def parsing(dat,stop_words):
    
    nacols = dat.isna()[['title', 'location', 'department', 'salary_range','description', 'requirements', 'benefits',
                      'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
                      'required_experience', 'required_education', 'industry', 'function.']].astype('int')
    
    numeric_cols = dat[['telecommuting', 'has_company_logo', 'has_questions']]
    # func to count words in document
    document_word_count = lambda document: len(document.split(' '))
    
    # count words in column
    columns = ["company_profile","description","requirements","benefits"]
    df = copy.deepcopy(dat[columns])
    for column in columns:
            df[(str(column) + "_length")] = dat[column].apply(lambda x: len(x) if x == x else 0)
    
    
    # salary column one hot
    category_1 = dat.salary_range.apply(salary_extract_first).apply(salary_category_first)
    category_2 = dat.salary_range.apply(salary_extract_second).apply(salary_category_second)
    
    salary_1_transform = pd.DataFrame.sparse.from_spmatrix(salary_1_onehot.transform(pd.DataFrame(category_1)))
    salary_2_transform = pd.DataFrame.sparse.from_spmatrix(salary_2_onehot.transform(pd.DataFrame(category_2)))
    
    # transform to one hot
    employment_type_transformed =  pd.DataFrame.sparse.from_spmatrix(employment_type_onehot.transform(dat[['employment_type']].fillna('NaN')))
    required_experience_transformed =  pd.DataFrame.sparse.from_spmatrix(required_experience_onehot.transform(dat[['required_experience']].fillna('NaN')))
    required_education_transformed =  pd.DataFrame.sparse.from_spmatrix(required_education_onehot.transform(dat[['required_education']].fillna('NaN')))
    industry_transformed =  pd.DataFrame.sparse.from_spmatrix(industry_onehot.transform(dat[['industry']].fillna('NaN')))
    function_transformed =  pd.DataFrame.sparse.from_spmatrix(function_onehot.transform(dat[['function.']].fillna('NaN')))
    
    
    return(pd.concat([nacols,salary_1_transform, salary_2_transform,df.iloc[:,4:],
                      employment_type_transformed, required_experience_transformed, required_education_transformed, industry_transformed,function_transformed,numeric_cols],axis = 1))
    

In [6]:
stop_words = {'a', 'the', 'of', 'in', 'for'}

In [7]:
dat = pd.read_csv("job_training_data.csv")

In [8]:
data = parsing(dat,stop_words)

In [32]:
X = data
y = dat.fraudulent
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)
rf = RandomForestClassifier(n_estimators = 200,max_features=None ,n_jobs=6)
rf.fit(X_train,y_train)
sklearn.metrics.confusion_matrix(y_test,rf.predict(X_test))

C:\Users\Sean\anaconda3\lib\site-packages\sklearn\utils\validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Sean\anaconda3\lib\site-packages\sklearn\utils\validation.py:515: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


array([[1535,    8],
       [  33,   33]], dtype=int64)

# Original method

In [33]:
import pandas as pd
import os
def parsing1(df):
    columns = ["company_profile","description","requirements","benefits"]
    one_hot_encoding_columns = ['employment_type', "required_education", "required_experience","function.", "industry"]
    length = []
    for column in columns:
        df[(str(column) + "_length")] = df[column].apply(lambda x: len(x) if x == x else 0)
    df = pd.get_dummies(df, columns = one_hot_encoding_columns)
    return df.iloc[:, 9:]

jobs = pd.read_csv("job_training_data.csv")
jobs = parsing1(jobs)
feature_matrix = jobs
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

feature_matrix = jobs
X = feature_matrix.drop("fraudulent", axis = 1)
y = feature_matrix.fraudulent
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
rfm = RandomForestClassifier()
rfm= RandomForestClassifier(n_estimators = 200 ,oob_score=True, n_jobs=6)
rfm.fit(X_train,y_train)
rfm.score(X_train,y_train)
y_pred_rfm=rfm.predict(X_test)

confusion_matrix(y_test, y_pred_rfm)

array([[1538,    5],
       [  36,   30]], dtype=int64)

# PCA

In [ ]:
data_normalized = preprocessing.scale(feature_matrix,with_mean=True,with_std=True)

In [ ]:
pca = PCA()
pca.fit(data_normalized)
components = pca.components_[0:np.where(pca.explained_variance_ratio_.cumsum() > 0.9)[0][0]+1,:]
pca_features = np.matmul(data_normalized,np.transpose(components))

# Random Forest

In [ ]:
X = feature_matrix
y = dat.fraudulent
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state=42)
rfm = RandomForestClassifier(n_estimators = 200,n_jobs=6)
rfm.fit(X_train,y_train)
y_pred_rfm=rfm.predict(X_test)

In [ ]:
sklearn.metrics.confusion_matrix(y_test,y_pred_rfm)